## **The Forward and Backward Passes of a Simple MLP / Neural Network**

In [1]:
import pickle, gzip, math, os, time, shutil
import numpy as np
import matplotlib as mpl
import torch
from torch import tensor
from fastcore.test import  test_close
from pathlib import Path

# Configs
torch.manual_seed(42)
mpl.rcParams['image.cmap'] = 'gray'
torch.set_printoptions(precision=2, linewidth=125, sci_mode=False)
np.set_printoptions(precision=2, linewidth=125)

# Path setup
path_data = Path('data')
path_gz = path_data/'mnist.pkl.gz'
with gzip.open(path_gz, 'rb') as f:
    ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
# Loading MNIST data as tensors
x_train, y_train, x_valid, y_valid = map(tensor, [x_train, y_train, x_valid, y_valid])

## Building the Foundations

### Basic Architecture

In [2]:
# Here n is the number of training examples and m is the number of pixels
n, m = x_train.shape
# C is the number of possible values of our outputs / digits
c = y_train.max() + 1
n, m, c

(50000, 784, tensor(10))

We will decide ahead of time what the number of our hidden **activations** or **nodes** for a single layer will be. Lets pick an arbitrary number...

In [3]:
# Number of hidden activations
nh = 50

As we have already established, we will be utilizing what we've learnt about matrix multiplications to get our output probabilities for each input row. We already have training data in a **`50,000x784`** matrix. For the linear function to work, we will need weights and biases.

In [4]:
# The weight matrix will contain 50000x50 random values
w1 = torch.randn(m, nh)
# Adding the biases for summation operations to create the linear function.
# Creating a matrix of 0s, one for each hidden activation
b1 = torch.zeros(nh)
# For layer two, we will start with nh inputs.
# But, we will stick with 1 output column to simplify the loss calcs. Which will be MSE instead of cross entropy.
w2 = torch.randn(nh, 1)
#  The sample applies for the bias i.e. sticking with a single output for simplicity.
b2 = torch.zeros(1)


In [5]:
# Creating a simple linear function for putting X through a single layer
def lin(x, w, b): return x@w + b

In [6]:
# Verifying the shape of our validation matrix
x_valid.shape

torch.Size([10000, 784])

In [7]:
# Passing the validation data through the linear function will result in a 10000x50 matrix
t = lin(x_valid, w1, b1)
t.shape

torch.Size([10000, 50])

In [8]:
# Passing the results through a ReLU
def relu(x): return x.clamp_min(0.)

In [9]:
t = relu(t)
t

tensor([[ 0.00, 11.87,  0.00,  ...,  5.48,  2.14, 15.30],
        [ 5.38, 10.21,  0.00,  ...,  0.88,  0.08, 20.23],
        [ 3.31,  0.12,  3.10,  ..., 16.89,  0.00, 24.74],
        ...,
        [ 4.01, 10.35,  0.00,  ...,  0.23,  0.00, 18.28],
        [10.62,  0.00, 10.72,  ...,  0.00,  0.00, 18.23],
        [ 2.84,  0.00,  1.43,  ...,  0.00,  5.75,  2.12]])

In [10]:
# A basic MLP which takes a mini-batch of data
def model(xb):
    l1 = lin(xb, w1, b1)
    l2 = relu(l1)
    return lin(l2, w2, b2)

In [11]:
# The model will now output a single column output
# Again, this is only for simplicity since we will be testing the model using MSE.
res = model(x_valid)
res.shape

torch.Size([10000, 1])

### Simple Loss Function: Mean Square Error

MSE is not a suitable loss function for multi-class classification. **We are over simplifying things for demonstration purposes only.**

In [12]:
res.shape, y_valid.shape

(torch.Size([10000, 1]), torch.Size([10000]))

In [13]:
(res - y_valid).shape

torch.Size([10000, 10000])

The calculation above is not correct. At the moment, based on broadcasting rules, each of our 10000 row items will be subtracted by the 10000 items in the array. PyTorch will automatically insert a unit x-axis in `y_valid`. This is not the shape of outputs that we desire!

For correct element-wise operations, we will get rid of the trailing unit axis in `res`

In [14]:
# One option is to use indexing to remove the trailing dimension
res[:, 0].shape

torch.Size([10000])

In [15]:
# Or we can use squeeze() to drop all trailing unit dimensions
res.squeeze().shape

torch.Size([10000])

In [16]:
# Demo: Adding arbitrary unit dimensions
test = res[None, :, None, None]
test.shape

torch.Size([1, 10000, 1, 1, 1])

In [17]:
# Using squeeze()
test.squeeze().shape

torch.Size([10000])

In [18]:
# Our results will now have the correct shape
(res.squeeze() - y_valid).shape

torch.Size([10000])

In [19]:
# Lets calculate our predictions for the training set
# Converting to float for MSE
y_train, y_valid = y_train.float(), y_valid.float()

preds = model(x_train)
preds.shape

torch.Size([50000, 1])

In [20]:
# Creating an MSE function
# Alternatively, we can also use output[:, 0] instead of squeeze as we have already established
def mse(output, target): return (output.squeeze() - target).pow(2).mean()

In [21]:
mse(preds, y_train)

tensor(4308.76)

### Calculating Our Gradients and Setting Up the Backward Pass

We will need our model to learn over a certain number of steps / epochs and this isn't going to be a one-off prediction. The neural network will need to calculate the gradient of the loss with regard to the weights of our model, in an iterative manner, to update the weights and biases of our model based on the direction of the gradient.

This is effectively an application of the chain rule. And when these derivatives are carried out in a matrix then that matrix is also called a Jacobian.

We can calculate derivatives symbolically using [SimPy](https://simpy.readthedocs.io/en/latest/topical_guides/simpy_basics.html).

In [22]:
from sympy import symbols, diff
x, y = symbols('x y')
diff(x**2, x)

2*x

In [23]:
diff(3*x**2 + 4*x + 9, x)

6*x + 4

In [24]:
# We will use the python debugger to explore the outputs of the functions
import pdb

In [25]:
# Updating the Linear Model function
def lin_grad(inp, out, w, b):
    # Grad of matmul with respect to input
    # Storing the gradients of our inputs, gradients of outputs wrt inputs
    inp.g = out.g @ w.t()
    #pdb.set_trace()
    # Gradient of the outputs wrt the weights will be sum of inputs*outputs
    w.g = (inp.unsqueeze(-1) * out.g.unsqueeze(1)).sum(0)
    # Gradient of the bias
    b.g = out.g.sum(0)

In [26]:
# Defining the func for our forward and backward pass
def forward_and_backward(inp, targ):
    # Forward pass:
    l1 = lin(inp, w1, b1)
    l2 = relu(l1)
    out = lin(l2, w2, b2)
    diff = out[:, 0] - targ # or unsqueeze()
    loss = diff.pow(2).mean()

    # Backward pass:
    # The attribute out.g will allow us to store the gradients in the layer itself
    # Also, due to the chain rule, we will have a number of intermediate calculations in the
    # backward pass.
    out.g = 2.*diff[:, None] / inp.shape[0] # Derivative of the loss wrt to the NN.
    lin_grad(l2, out, w2, b2) # Calling intermediate calc from the forward pass.
    l1.g = (l1 > 0).float() * l2.g # Another intermediate call
    lin_grad(inp, l1, w1, b1)

In the code block above, the **Backward Pass** components are :
- `out.g = 2.*diff[:, None] / inp.shape[0]` calculates the gradient of the loss wrt `out` and comes from the formula for derivative of the Mean Squared Error (MSE) function.
- `lin_grad(l2, out, w2, b2)` compute the gradients for the second layer.
- `l1.g = (l1 > 0).float() * l2.g` computes the gradient of ReLU activation which is 1 for positive inputs and 0 for negative inputs. `l1 > 0` creates a mask of 1s and 0s.

In [27]:
# Calling forward and backward
forward_and_backward(x_train, y_train)

This section contains some really "clunky" code and could do with some optimization.

In [28]:
# Save for testing
def get_grad(x): return x.g.clone()

chks = w1, w2, b1, b2, x_train
grads = w1g, w2g, b1g, b2g, ig = tuple(map(get_grad, chks))

In [29]:
# We can use PyTorch autograd to check our results
def mkgrad(x): return x.clone().requires_grad_(True)

ptgrads = w12, w22, b12, b22, xt2 = tuple(map(mkgrad, chks))

In [30]:
def forward(inp, targ):
    l1 = lin(inp, w12, b12)
    l2 = relu(l1)
    out = lin(l2, w22, b22)
    return mse(out, targ)

In [31]:
loss = forward(xt2, y_train)
loss.backward()

In [32]:
for a, b in zip(grads, ptgrads): test_close(a, b.grad, eps=0.01)

## Refactor the Model

### Layers as Classes

We can refactor our code to include classes with callable objects to minimize manual tinkering with the various components of the code.

In [33]:
class Relu():
    # This class will calculate ReLU for each forward and backward pass on top of storing the outputs
    def __call__(self, inp): # Forward pass
        self.inp = inp
        self.out = inp.clamp_min(0.)
        return self.out

    def backward(self):
        # Calling the stored input gradients
        self.inp.g = (self.inp > 0).float() * self.out.g

In [34]:
class Lin():
    # Storing the weights and biases for Linear calcs
    def __init__(self, w, b):
        self.w, self.b = w, b

    def __call__(self, inp):
        self.inp = inp
        # Calling the linear function from before. This approach allows us to stack funcs within calls.
        self.out = lin(inp, self.w, self.b)
        return self.out

    def backward(self):
        self.inp.g = self.out.g @ self.w.t() # .T can be used interchangeably
        self.w.g = self.inp.t() @ self.out.g
        self.b.g = self.out.g.sum(0)

In [35]:
class Mse():
    def __call__(self, inp, targ):
        # Storing the input and the target
        self.inp, self.targ = inp, targ
        self.out = mse(inp, targ)
        return self.out

    def backward(self):
        self.inp.g = 2. * (self.inp.squeeze() - self.targ).unsqueeze(-1) / self.targ.shape[0]

In [36]:
class Model():
    def __init__(self, w1, b1, w2, b2):
        # Storing the model's layers i.e. instances of classes
        self.layers = [Lin(w1, b1), Relu(), Lin(w2, b2)]
        self.loss = Mse()

    def __call__(self, x, targ):
        # Calling each layer
        for l in self.layers: x = l(x)
        # HuggingFace puts their loss inside the forward pass
        # FastAI, and others, use separate functions for loss calculations
        return self.loss(x, targ)

    def backward(self):
        # loss is MSE() which already has inputs and targets stored.
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [37]:
model = Model(w1, b1, w2, b2)

In [38]:
loss = model(x_train, y_train)

In [39]:
model.backward()

In [40]:
# Each of the gradients were stored earlier
test_close(w2g, w2.g, eps=0.01)
test_close(b2g, b2.g, eps=0.01)
test_close(w1g, w1.g, eps=0.01)
test_close(b1g, b1.g, eps=0.01)
test_close(ig, x_train.g, eps=0.01)

## Module.forward()

In the previous section, our code has numerous repititions for e.g. `self.inp` and `self.out` are being called a bunch of times.

The Module class will allow us to simplify the code even further.

In [41]:
class Module(): # Module will be an inherited class
    def __call__(self, *args): # *args stores arguments in a list
        self.args = args
        self.out = self.forward(*args)
        return self.out

    def forward(self): raise Exception('not implemented')

    def backward(self): self.bwd(self.out, *self.args) 

    def bwd(self): raise Exception('not implemented')

In the block above `*self.args` unpacks elements of `self.args` (which is expected to be an iterable like a list or tuple) as arguments. 

With the object inheritance, we are able to clean up the Relu() module.

In [42]:
class Relu(Module):
    def forward(self, inp): return inp.clamp_min(0.)

    def bwd(self, out, inp): inp.g = (inp > 0).float() * out.g

Same for Lin() and MSE().

In [43]:
class Lin(Module):
    def __init__(self, w, b): self.w, self.b = w, b

    def forward(self, inp): return inp@self.w + self.b

    def bwd(self, out, inp):
        inp.g = self.out.g @ self.w.t()
        self.w.g = inp.t() @ self.out.g
        self.b.g = self.out.g.sum(0)

Sometimes, at the cost of some memory, we can further optimize calculations by storing partial results.

Notice that `(inp.squeeze() - targ)` is being called twice and can be stored in an object and called directly.

In [49]:
class Mse(Module):
    def forward(self, inp, targ): return (inp.squeeze() - targ).pow(2).mean()

    def bwd(self, out, inp, targ): inp.g = 2.*(inp.squeeze() - targ).unsqueeze(-1) / targ.shape[0]

In [45]:
model = Model(w1, b1, w2, b2)

In [46]:
loss = model(x_train, y_train)

In [47]:
model.backward()

In [48]:
test_close(w2g, w2.g, eps=0.01)
test_close(b2g, b2.g, eps=0.01)
test_close(w1g, w1.g, eps=0.01)
test_close(b1g, b1.g, eps=0.01)
test_close(ig, x_train.g, eps=0.01)

## Autograd Using PyTorch

Having built out the classes in previous sections, we've satisfied the course's conditions regarding abstractions.

PyTorch already has things built for us!

In [50]:
from torch import nn
import torch.nn.functional as F

In [51]:
# Using PyTorch's Module class.
class Linear(nn.Module):
    def __init__(self, n_in, n_out):
        super().__init__()
        # As before, lets generate our random weights
        self.w = torch.randn(n_in, n_out).requires_grad_()
        self.b = torch.zeros(n_out).requires_grad_()

    def forward(self, inp): return inp@self.w + self.b

We won't need to create `backwards()` since PyTorch will handle the storage of derivatives for us using the chain rule.

In [55]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [Linear(n_in, nh), nn.ReLU(), Linear(nh, n_out)]

    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        # Using PyTorch's mse_loss()
        return F.mse_loss(x, targ[:, None])

In [53]:
model = Model(m, nh, 1)
loss = model(x_train, y_train)
loss.backward()

In [54]:
l0 = model.layers[0]
l0.b.grad

tensor([-19.60,  -2.40,  -0.12,   1.99,  12.78, -15.32, -18.45,   0.35,   3.75,  14.67,  10.81,  12.20,  -2.95, -28.33,
          0.76,  69.15, -21.86,  49.78,  -7.08,   1.45,  25.20,  11.27, -18.15, -13.13, -17.69, -10.42,  -0.13, -18.89,
        -34.81,  -0.84,  40.89,   4.45,  62.35,  31.70,  55.15,  45.13,   3.25,  12.75,  12.45,  -1.41,   4.55,  -6.02,
        -62.51,  -1.89,  -1.41,   7.00,   0.49,  18.72,  -4.84,  -6.52])

Et Voila!